In [2]:
from nltk.tokenize.casual import casual_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import re
import pandas as pd
import string
import os
base = "../../data/clean/"
pd.set_option('display.max_rows', 150)

In [87]:
tweetdf = pd.read_csv(base+'twitter_17_features.csv')
tweetdf = tweetdf.sort_values(by='lga')
tweetdf.head()

,lga,instagrammers_ratio,replyers_per_capita,total_tweeters,link_tweets_ratio,Polr_Neg_A_pcapita,Polr_Neg_N_pcapita,Polr_Neg_E_pcapita,Polr_Neg_M_pcapita,Night_pcapita,Morning_pcapita,Afternoon_pcapita,Evening_pcapita,Prof_exists_pcapita,tr_tweeters,ja_tweeters,in_tweeters,tl_tweeters
2,Albury,0.339355,0.006304,775,0.316016,-0.002669,-0.007370,-0.004476,-0.008072,0.128044,0.152263,0.045873,0.083836,0.024101,4,3,26,29
108,Armidale Regional,0.375817,0.003362,306,0.514424,-0.001420,-0.001944,-0.001170,-0.004177,0.062039,0.079527,0.025773,0.036300,0.004584,0,1,5,5
99,Ballina,0.594556,0.003853,698,0.550613,-0.000477,-0.001150,-0.000811,-0.001753,0.040608,0.051400,0.010625,0.028093,0.002560,0,4,10,10
56,Bathurst Regional,0.476974,0.004383,608,0.444390,-0.000543,-0.002403,-0.001372,-0.004298,0.047240,0.063051,0.010242,0.028814,0.007676,0,1,4,7
85,Bayside,0.042857,0.001538,770,0.265229,-0.000143,-0.000496,-0.000353,-0.000652,0.009817,0.012848,0.001890,0.007529,0.001057,1,6,10,14


In [16]:
import sys
sys.path.append("../..")
from src.data import utils

target = utils.load_target()

In [88]:
tweetdf = tweetdf.set_index("lga")

In [82]:
import shapefile
from shapely.geometry import shape, Point, mapping
import numpy as np
import pandas as pd

raw = "../../data/raw/"
clean = "../../data/processed/"
filename = "2016_LGA_SHAPE/LGA_2016_AUST"

sf = shapefile.Reader(raw+filename)

nswshapes = list(map(lambda x: shape(x.__geo_interface__), sf.shapes()[:130]))
centroids = list(map(lambda x: shape(x.__geo_interface__).centroid, sf.shapes()[:130]))
df = pd.DataFrame(sf.records())
nswdf = df[df[4] =="New South Wales"]
# drop "No usual address" - not a useful LGA
nswdf = nswdf[nswdf[0] != "LGA19499"]
nswdf.columns = ["code", "codenum", "name", "unknown", "state", "unknown2"]
nswdf["clean_name"] = nswdf["name"].str.extract('([^\\(]*)', expand=False).str.strip()


In [60]:
#amalgamations on shpfile
amalgamations = {"Botany Bay":"Bayside",
                 "Rockdale":"Bayside",
                 "Greater Hume Shire": "Greater Hume",
                 "Gundagai": 'Cootamundra-Gundagai',
                 'Western Plains Regional': "Dubbo Regional",
                 'Unincorporated NSW': 'Unincorporated Far West',
                 
                }

for x in amalgamations.keys():
    nswdf.loc[nswdf["clean_name"] == x, "clean_name"] =  amalgamations[x]

In [37]:
import json
lga_json = list(map(lambda x: x.__geo_interface__, sf.shapes()[:130]))

geojson = {'type': 'FeatureCollection',
            'features': [{'type':'Feature', 'geometry':x, 'properties': {'name':nswdf.iloc[i]['clean_name']}} for i, x in enumerate(lga_json)]
          }

#with open('geojson.json','w') as f:
#    json.dump(geojson, f)

In [7]:
aggdfs = []
cols = ['tr_tweeters','ja_tweeters','in_tweeters','tl_tweeters']


In [89]:
joindf = nswdf.merge(tweetdf, left_on="clean_name", right_index=True, how="outer").fillna(0)
tweetdf= joindf.set_index("clean_name")

In [10]:
def colorshp(col, cmax, cmin):
    portion = (col-cmin)
    if portion > 0:
        portion = np.log(portion)
    point = portion/np.log((cmax - cmin))
    #print(point)
    strength = int(point * 255)
    alpha = point
    return 'rgba({0},0,0,{1:.2f})'.format(strength,alpha)

In [76]:
=

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objs as go
from plotly.graph_objs import *

app = dash.Dash()
os.environ['WERKZEUG_RUN_MAIN'] = 'true'

data = []
for i, tok in enumerate(cols):
    #lga boundaries
    lga_layers = list(map(lambda x: { 
                    'sourcetype' : "geojson",
                    'source': x,
                    'type'   : 'fill',
                    'color': colorshp(tweetdf.loc[x["properties"]["name"]][tok], tweetdf[tok].max(),0),
                    }, geojson['features']))

    #lga labels
    labels = dict(type='scattermapbox',
                lon=[x.x for x in centroids],
                lat=[x.y for x in centroids],
                mode='text',
                text=list(nswdf.clean_name),
                hoverinfo = 'none',
                 )
    #counts for hover
    counts = dict(type='scattermapbox',
                lon=[x.x for x in centroids],
                lat=[x.y for x in centroids],
                mode='none',
                text = tweetdf[tok].loc[list(nswdf.clean_name)],
                hoverinfo = 'text',
                 )
    data.append({
            "data": [labels, counts],
            "layout": go.Layout(
                hovermode = "closest",
                margin = dict(l = 0, r = 0, t = 0, b = 0),
                mapbox = dict(
                    layers = lga_layers,
                    accesstoken = "pk.eyJ1IjoiZGVzdWx0aXIiLCJhIjoiY2pmZDN4M2FlMTEzYjJ3bWlyc2VhaDkxaiJ9.VvezwDLTQlh2GVAEsauwlg",
                    bearing = 0,
                    center = dict(lat=-32.662833, lon=147.031917),
                    style = "light",
                    pitch = 0,
                    zoom = 5.5,
                )
            )
        })
    
app.layout = html.Div(children=[
    html.H4(children='Tweeters by LGA'),
    dcc.Slider(
        id='slider',
        min=0,
        max=len(cols),
        step=1,
        value=0,
        marks={i:cols[i] for i in range(len(cols))},
    ),
    html.Br(),
    dcc.Graph(
        id = "mapbox",
        #animate=True,
        style = {"height": 750}
    )])

@app.callback(
    dash.dependencies.Output('mapbox', 'figure'),
    [dash.dependencies.Input('slider', 'value')])
def update_graph(value):
    print(value)
    return {
        'data': data[value]['data'],
        'layout': data[value]['layout']
    }

app.run_server(debug=False, port=8012)

 * Running on http://127.0.0.1:8012/ (Press CTRL+C to quit)
127.0.0.1 - - [30/May/2018 16:03:09] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [30/May/2018 16:03:10] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [30/May/2018 16:03:10] "GET /_dash-dependencies HTTP/1.1" 200 -


0


127.0.0.1 - - [30/May/2018 16:03:13] "GET /favicon.ico HTTP/1.1" 200 -
127.0.0.1 - - [30/May/2018 16:03:14] "POST /_dash-update-component HTTP/1.1" 200 -


1


127.0.0.1 - - [30/May/2018 16:06:31] "POST /_dash-update-component HTTP/1.1" 200 -


3


127.0.0.1 - - [30/May/2018 16:07:17] "POST /_dash-update-component HTTP/1.1" 200 -


In [11]:
#
